# Финальный проект - A/B тестирование

## Введение

**Описание проекта**

Наша задача — провести оценку результатов A/B-теста. В нашем распоряжении есть датасет с действиями пользователей, техническое задание и несколько вспомогательных датасетов. Нам надо:

1. Оценить корректность проведения теста.
2. Проанализировать результаты теста.

Чтобы оценить корректность проведения теста, проверим:

1. Пересечение тестовой аудитории с конкурирующим тестом.
2. Совпадение теста и маркетинговых событий, другие проблемы временных границ теста.

**Техническое задание**

1. Название теста: `recommender_system_test`.
2. Группы: А — контрольная, B — новая платёжная воронка.
3. Дата запуска: `2020-12-07`.
4. Дата остановки набора новых пользователей: `2020-12-21`.
5. Дата остановки: `2021-01-04`.
6. Фудитория: 15% новых пользователей из региона EU.
7. Назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы.
8. Ожидаемое количество участников теста: 6000.
9. Ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%: конверсии в просмотр карточек товаров — событие product_page, просмотры корзины — product_cart, покупки — purchase. 

**Описание данных**

`ab_project_marketing_events.csv` — календарь маркетинговых событий на 2020 год.

Структура файла:
1. `name` — название маркетингового события.
2. `regions` — регионы, в которых будет проводиться рекламная кампания.
3. `start_dt` — дата начала кампании.
4. `finish_dt` — дата завершения кампании.
   
`final_ab_new_users.csv` — пользователи, зарегистрировавшиеся с 7 по 21 декабря 2020 года.

Структура файла:
1. `user_id` — идентификатор пользователя.
2. `first_date` — дата регистрации.
3. `region` — регион пользователя.
4. `device` — устройство, с которого происходила регистрация.

`final_ab_events.csv` — действия новых пользователей в период с 7 декабря 2020 по 4 января 2021 года.

Структура файла:
1. `user_id` — идентификатор пользователя.
2. `event_dt` — дата и время покупки.
3. `event_name` — тип события.
4. `details` — дополнительные данные о событии. Например, для покупок, purchase, в этом поле хранится стоимость покупки в долларах.

`final_ab_participants.csv` — таблица участников тестов.

Структура файла:
1. `user_id` — идентификатор пользователя.
2. `ab_test` — название теста.
3. `group` — группа пользователя.

**План работы**

1. Исследование данных: преобразование типов, поиск пропущенных значений и дубликатов.
2. Оценить корректность проведения теста. Проверить:
   1. Соответствие данных требованиям технического задания. Корректность всех пунктов технического задания.
   2. Время проведения теста: оно не должно совпадать с маркетинговыми и другими активностями.
   3. Аудиторию теста. Удостовериться, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно. Проверить равномерность распределения по тестовым группам и правильность их формирования.
3. Провести исследовательский анализ данных:
   1. Количество событий на пользователя одинаково распределены в выборках?
   2. Как число событий в выборках распределено по дням?
   3. Как меняется конверсия в воронке в выборках на разных этапах?
   4. Какие особенности данных нужно учесть, прежде чем приступать к A/B-тестированию?
4. Оценить результаты A/B-тестирования:
   1. Что можно сказать про результаты A/В-тестирования?
   2. Проверить статистическую разницу долей z-критерием.
5. Сделать общее заключение о корректности проведения теста.

In [1]:
import pandas as pd
import plotly.express as px
from scipy import stats as st
from IPython.display import display

# Save raw datasets in case we need them
try:
    raw_users = pd.read_csv('final_ab_new_users.csv')
    raw_tests = pd.read_csv('final_ab_participants.csv')
    raw_events = pd.read_csv('final_ab_events.csv')
    raw_marketing = pd.read_csv('ab_project_marketing_events.csv')
except:
    raw_users = pd.read_csv('/datasets/final_ab_new_users.csv')
    raw_tests = pd.read_csv('/datasets/final_ab_participants.csv')
    raw_events = pd.read_csv('/datasets/final_ab_events.csv')
    raw_marketing = pd.read_csv('/datasets/ab_project_marketing_events.csv')

# Constants, which we will need later
FIG_WIDTH = 8
FIG_HEIGHT = 5


## Предобработка данных

Для начала проверим какие данные нам достались. Нас интересуют:

1. Названия колонок.
2. Неподходящие под тип данных значения в колонках.
3. Артефакты в данных и повторяющиеся строки.
4. Пропущенные значения.
5. Дополнительные колонки и типы данных.

Пойдем по порядку.

In [2]:
# Check columns
for dataset in [raw_users, raw_tests, raw_events, raw_marketing]:
    print('-' * 50)
    dataset.info()

--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column   

Название колонок уже заданы нормально. У нас есть много пропущенных занчений в датасете `raw_events`, но это может быть связано с тем, как были собраны данные. Мы на это посмотрим в следующих секциях. Наконец, надо будет задать типы данных.

Посмотрим, что за данные у нас внутри этих таблиц:

In [12]:
(raw_users
 .fillna('NA')
 .pivot_table(index=['region','device'], values='user_id', aggfunc='count')
 .reset_index()
 .rename(columns={'user_id':'count'})
)

,region,device,count
0,APAC,Android,1401
1,APAC,Mac,316
2,APAC,PC,803
3,APAC,iPhone,633
4,CIS,Android,1413
5,CIS,Mac,310
6,CIS,PC,776
7,CIS,iPhone,656
8,EU,Android,20629
9,EU,Mac,4575


In [13]:
(raw_tests
 .fillna('NA')
 .pivot_table(index=['group','ab_test'], values='user_id', aggfunc='count')
 .reset_index()
 .rename(columns={'user_id':'count'})
)

,group,ab_test,count
0,A,interface_eu_test,5831
1,A,recommender_system_test,3824
2,B,interface_eu_test,5736
3,B,recommender_system_test,2877


In [14]:
(raw_events
 .fillna('NA')
 .pivot_table(index=['event_name','details'], values='user_id', aggfunc='count')
 .reset_index()
 .rename(columns={'user_id':'count'})
)

,event_name,details,count
0,login,NA,189552
1,product_cart,NA,62462
2,product_page,NA,125563
3,purchase,4.99,46362
4,purchase,9.99,9530
5,purchase,99.99,5631
6,purchase,499.99,1217


In [15]:
raw_marketing

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11
5,Black Friday Ads Campaign,"EU, CIS, APAC, N.America",2020-11-26,2020-12-01
6,Chinese New Year Promo,APAC,2020-01-25,2020-02-07
7,Labor day (May 1st) Ads Campaign,"EU, CIS, APAC",2020-05-01,2020-05-03
8,International Women's Day Promo,"EU, CIS, APAC",2020-03-08,2020-03-10
9,Victory Day CIS (May 9th) Event,CIS,2020-05-09,2020-05-11


Похоже, все на своих местах, кроме `raw_marketing` таблицы. В колонке `regions` у нас записаны значения через запятую, что может вызывать проблемы.

Колонка `details` в таблице `raw_events`, видимо, показывает стоимость покупки. Для событий, которые не соответствуют покупкам, эта колонка будет пустой (ожидаемо).

Наконец, в таблице `raw_tests` наши пользователи из групп А и В принимали участия в двух экспериментах. Нам надо будет это учесть в анализе дальше.

Причешем наши датасеты.

In [18]:
df_users = raw_users.copy().astype({'first_date': 'datetime64[D]'})
df_tests = raw_tests.copy()
df_events = raw_events.copy().astype({'event_dt': 'datetime64[s]', 'details': 'float'})
df_marketing = raw_marketing.copy().astype({'start_dt': 'datetime64[D]', 'finish_dt': 'datetime64[D]'})